In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from datetime import date, time, timedelta
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import ignite
from sklearn.metrics import r2_score
import seaborn as sn
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
# Helper functions for manipulating dates
def handle_weekends(date):
    # Makes sure day is not weekend since weekends doesnt contain stock prices
    if date.weekday() == 5:
        return date -  timedelta(days=1)
    elif date.weekday() == 6:
        return date + timedelta(days=1)
    else:
        return date

def add_year(date):
    date = date + timedelta(days=365)
    return handle_weekends(date)    
    
def sub_year(date):
    date = date - timedelta(days=365)
    return handle_weekends(date)    
    
PRICE_FILE = "../data/prices3.csv"
price_df = pd.read_csv(PRICE_FILE)
price_df['date'] = pd.to_datetime(price_df['date'], format='%Y%m%d')

FILE_NAME1 = "../data/sample_company_data3.csv"
df1 = pd.read_csv(FILE_NAME1)

df1['public_date'] = pd.to_datetime(df1['public_date'], format='%Y%m%d')
df1['date_after_year'] = df1['public_date'].apply(add_year)
df1['date_before_year'] = df1['public_date'].apply(sub_year)

df1['public_date'] = df1['public_date'].apply(handle_weekends)

data_start_price = price_df.rename(columns={'PRC': 'start_price'})
data_end_price = price_df.rename(columns={'PRC': 'end_price'})
data_momentum = price_df.rename(columns={'PRC': 'price_year_before'})
    
# Merge the prices with the financial ratios to make the final dataset
new_df = pd.merge(df1, data_start_price, how='left', left_on=['public_date', 'permno'], right_on=['date', 'PERMNO'])
new_df = pd.merge(new_df, data_end_price, how='left', left_on=['date_after_year', 'permno'], right_on=['date', 'PERMNO'])
new_df = pd.merge(new_df, data_momentum, how='left', left_on=['date_before_year', 'permno'], right_on=['date', 'PERMNO'])

new_df = new_df.drop(['gvkey', 'date_after_year', 'PERMNO_x', 'date_x', 'COMNAM_x', 'CUSIP_x','PERMNO_y', 'date_y', 'COMNAM_y', 'CUSIP_y', 'PERMNO', 'date', 'COMNAM'], axis=1).fillna(0)


In [4]:
new_df = new_df[new_df.start_price != 0]
new_df = new_df[new_df.price_year_before != 0]
new_df = new_df[new_df.end_price != 0]

new_df['start_price'] = new_df['start_price'].abs()
new_df['price_year_before'] = new_df['price_year_before'].abs()
new_df['end_price'] = new_df['end_price'].abs()


new_df['growth'] = (((new_df['end_price']-new_df['start_price']) / new_df['start_price']))
new_df['divyield'] = new_df['divyield'].str.rstrip('%').astype('float').fillna(0)
new_df['momentum'] = (((new_df['start_price']-new_df['price_year_before']) / new_df['price_year_before']))


In [5]:
new_df = new_df.sort_values(by=['public_date']).fillna(0)

In [11]:
new_df.to_csv("financialratios.zip", compression='zip')